<a href="https://www.kaggle.com/code/dewilliams/multi-agent-life-advisory-board?scriptVersionId=232520468" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Multi-Agent Life Advisor: Interactive Advisory Board
## Use Case
This project creates an interactive "board of advisors" with Career and Money agents, using LangGraph and tools to provide personalized, actionable advice tailored to specific user goals—or to help articulate goals when users are unsure.

## Problem
People often need tailored guidance to achieve specific career or financial goals but may struggle to define those goals clearly, limiting the effectiveness of generic advice.

## Solution
We use:
1. **Agents**: Career and Money advisors collaborate via LangGraph in an interactive chat, powered by configurable LLMs (cloud or local), to offer goal-specific advice or assist in goal-setting.
2. **Retrieval-Augmented Generation (RAG)**: Grounds advice via a `@tool`-annotated function, inspired by *Principles* (Dalio), *The Intelligent Investor* (Graham), and the FIRE Toolbox.
3. **Structured Output**: Returns advice in JSON when finalized, with conversational flexibility.
4. **Evaluation**: Optionally collects user ratings (1-5) to assess advice quality, saved to a JSON file for potential future training.

## How to Use
1. **Setup**: Run the setup cell to install dependencies. Ensure `GOOGLE_API_KEY` is set in Kaggle Secrets for Gemini LLM.
2. **Configuration**: Adjust `CONFIG` below to toggle evaluation (`ENABLE_EVALUATION`), swap LLMs (`MODEL_CONFIG`), and customize behavior.
3. **Run**: Execute the notebook to start the chat. Type goals or questions, rate advice if enabled, and use 'q' to quit and see/download ratings.
4. **Download Ratings**: After quitting, run the download cell to access `ratings.json`.

## Use the API

Start by installing and importing the Gemini API Python SDK.

Occasionally, this block throws a pip warning on the 1st run. If you run it a 2nd time, it runs fine with no warning.

In [2]:
# Install langgraph and the packages used in this project.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7' 'numpy==1.26.4' 'llama-cpp-python==0.2.23'  # Added llama-cpp-python for local LLM option

In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'0.2.2'

### Automated retry

In [4]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google import genai
from google.genai import types
from IPython.display import Markdown, HTML, display
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

genai.__version__

'0.2.2'

## Setup

In [17]:
## Setup
import json
import numpy as np
import os
from langgraph.graph import StateGraph, END
from typing import Annotated, Dict, List, TypedDict
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from llama_cpp import Llama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from IPython.display import Markdown, display, FileLink
from kaggle_secrets import UserSecretsClient
from datetime import datetime

# Configuration
CONFIG = {
    "ENABLE_EVALUATION": True,  # Toggle rating prompts (True/False)
    "MODEL_CONFIG": {
        "career": {
            "type": "gemini",
            "model": "gemini-1.5-flash",
            "api_key": UserSecretsClient().get_secret("GOOGLE_API_KEY"),
            "max_tokens": 500
        },
        "money": {
            "type": "gemini",
            "model": "gemini-1.5-flash",
            "api_key": UserSecretsClient().get_secret("GOOGLE_API_KEY"),
            "max_tokens": 500
        }
        # Example for local LLM (uncomment to use):
        # "career": {
        #     "type": "local",
        #     "path": "mistral-7b-instruct-v0.2.Q4_K_M.gguf",
        #     "n_gpu_layers": 35,
        #     "n_ctx": 2048,
        #     "max_tokens": 500
        # }
    }
}

# Load LLMs based on config
def load_llm(advisor_type):
    cfg = CONFIG["MODEL_CONFIG"][advisor_type]
    if cfg["type"] == "gemini":
        return ChatGoogleGenerativeAI(model=cfg["model"], google_api_key=cfg["api_key"], max_tokens=cfg["max_tokens"])
    elif cfg["type"] == "local":
        return Llama(model_path=cfg["path"], n_gpu_layers=cfg["n_gpu_layers"], n_ctx=cfg["n_ctx"])
    else:
        raise ValueError(f"Unsupported LLM type: {cfg['type']}")

llm_career = load_llm("career")
llm_money = load_llm("money")

# Load corpus globally
corpus = [
    {"text": "Set clear, audacious goals and iterate based on feedback to advance your career.", "domain": "career", "source": "*Principles* by Ray Dalio"},
    {"text": "Embrace disruption by learning new skills to stay competitive in your field.", "domain": "career", "source": "*The Innovator’s Dilemma* by Clayton Christensen"},
    {"text": "Manage career risks by diversifying your skills and experiences.", "domain": "career", "source": "*The Most Important Thing* by Howard Marks"},
    {"text": "View your career as a journey—overcome challenges to grow into a leader.", "domain": "career", "source": "*The Hero with a Thousand Faces* by Joseph Campbell"},
    {"text": "Seek untapped career paths where competition is low and opportunity is high.", "domain": "career", "source": "*Blue Ocean Strategy* by W. Chan Kim & Renée Mauborgne"},
    {"text": "Build a strong network and reputation to climb the career ladder.", "domain": "career", "source": "*Alexander Hamilton* by Ron Chernow"},
    {"text": "Set ambitious career goals and work tirelessly to achieve them.", "domain": "career", "source": "*Think Big* by Ben Carson"},
    {"text": "Take initiative—volunteer for tough projects to prove leadership.", "domain": "career", "source": "*How to Become CEO* by Jeffrey J. Fox"},
    {"text": "Stay strategic—anticipate industry shifts to position yourself ahead.", "domain": "career", "source": "*The Kill Chain* by Christian Brose"},
    {"text": "Learn in-demand skills like data analysis to boost your career trajectory.", "domain": "career", "source": "Web trends (e.g., X posts on tech skills)"},
    {"text": "Invest in low-cost index funds for steady, long-term wealth growth.", "domain": "money", "source": "*The Intelligent Investor* by Benjamin Graham"},
    {"text": "Understand supply and demand to make smarter financial decisions.", "domain": "money", "source": "*Basic Economics* by Thomas Sowell"},
    {"text": "Balance risk and reward—don’t chase high returns without research.", "domain": "money", "source": "*The Most Important Thing* by Howard Marks"},
    {"text": "Use the debt snowball method—pay off smallest debts first to build momentum.", "domain": "money", "source": "FIRE Toolbox (Dave Ramsey’s Baby Steps)"},
    {"text": "Save 50% of your income by cutting unnecessary expenses like dining out.", "domain": "money", "source": "FIRE Toolbox (Mr. Money Mustache)"},
    {"text": "Aim for a 4% withdrawal rate in retirement with a diversified portfolio.", "domain": "money", "source": "FIRE Toolbox (Investing Simplified)"},
    {"text": "Build ‘f-you money’—enough savings to walk away from a bad job.", "domain": "money", "source": "FIRE Toolbox (JL Collins)"},
    {"text": "Live frugally—spend less than you earn to accelerate savings.", "domain": "money", "source": "FIRE Toolbox (Mr. Money Mustache)"},
    {"text": "Allocate 20% of income to savings to hit financial goals faster.", "domain": "money", "source": "FIRE Toolbox (Financial Samurai)"},
    {"text": "Start a side hustle to boost income and savings potential.", "domain": "money", "source": "FIRE Toolbox (Path to Passive Income)"}
]
corpus_texts = [entry["text"] for entry in corpus]

print("Notebook setup complete with LangGraph, tools, and configured LLMs loaded.")

## System Instructions
CAREER_SYSINT = (
    "You are a Career Advisor (CA) on an interactive advisory board. "
    "Focus on providing actionable advice to help the user achieve specific career goals (e.g., jobs, skills, business ventures). "
    "If the user hasn’t specified a goal, ask probing questions to help them articulate one (e.g., 'What career outcome are you aiming for?' or 'Are you looking to advance, switch fields, or start something new?'). "
    "Use the 'retrieve_context' tool for corpus insights when needed by calling it explicitly. "
    "Keep responses concise and conversational, starting with 'Career Advisor (CA):'. "
    "If the conversation shifts to finance-related topics (e.g., savings, investments, net worth), invite the Money Advisor (MA) once by saying: 'This seems like a financial question—let me bring in my colleague, the Money Advisor (MA), to assist.' and then defer unless prompted again. "
    "If asked to finalize advice, provide a JSON object with 'career', 'money' (empty if not relevant), 'combined', and 'timestamp'."
)

MONEY_SYSINT = (
    "You are a Money Advisor (MA) on an interactive advisory board. "
    "Focus on providing actionable advice to help the user achieve specific financial goals (e.g., savings, investments, net worth). "
    "If the user hasn’t specified a goal, ask probing questions to help them articulate one (e.g., 'What financial outcome are you targeting?' or 'Are you aiming to save, invest, or manage debt?'). "
    "Use the 'retrieve_context' tool for corpus insights when needed by calling it explicitly. "
    "Keep responses concise and conversational, starting with 'Money Advisor (MA):'. "
    "If the conversation shifts to career-related topics (e.g., jobs, skills, business ventures), invite the Career Advisor (CA) once by saying: 'This seems like a career question—let me bring in my colleague, the Career Advisor (CA), to assist.' and then defer unless prompted again. "
    "If asked to finalize advice, provide a JSON object with 'career' (empty if not relevant), 'money', 'combined', and 'timestamp'."
)

Notebook setup complete with LangGraph, tools, and configured LLMs loaded.


## Tools

In [18]:
@tool
def retrieve_context(query: str, domain: str) -> str:
    """Retrieve relevant context from the corpus based on a query and domain (career or money)."""
    query_words = set(query.lower().split())
    best_match = max(corpus, key=lambda x: len(query_words.intersection(x["text"].lower().split())) if x["domain"] == domain else -1)
    return f"{best_match['text']} (Source: {best_match['source']})"

## Build Interactive Chat with LangGraph

In [19]:
## Build Interactive Chat with LangGraph
# Define state
class AdvisorState(TypedDict):
    career_goal: str
    money_goal: str
    question: str
    career_context: str
    money_context: str
    combined_advice: Dict
    messages: Annotated[List, add_messages]
    finished: bool
    active_advisors: List[str]  # Track active advisors (CA, MA, or both)
    ratings: List[int]  # Store user ratings for evaluation

# Nodes
def chatbot(state: AdvisorState) -> AdvisorState:
    if not state["messages"]:
        state["active_advisors"] = ["CA"]  # Start with CA
        state["ratings"] = []  # Initialize ratings list
        return state | {"messages": [AIMessage(content="Welcome! I’m your Career Advisor (CA), and my colleague is your Money Advisor (MA). How can we assist you today?")]}
    
    last_msg = state["messages"][-1].content.lower()
    # Update state based on user input
    if "career goal" in last_msg:
        state["career_goal"] = last_msg.split("career goal")[-1].split(".")[0].strip()
    elif "money goal" in last_msg:
        state["money_goal"] = last_msg.split("money goal")[-1].split(".")[0].strip()
    elif "what should i focus on" in last_msg or "how can i" in last_msg:
        state["question"] = last_msg.strip()
    
    # Determine advisor and LLM based on topic
    sysint = ""
    llm_to_use = None
    invite_needed = False
    if "career" in last_msg or "job" in last_msg or "business" in last_msg or "pivot" in last_msg:
        sysint = CAREER_SYSINT
        llm_to_use = llm_career
        if "CA" not in state["active_advisors"] and "MA" in state["active_advisors"]:
            invite_needed = True
            state["active_advisors"].append("CA")
    elif "money" in last_msg or "finance" in last_msg or "net worth" in last_msg or "sav" in last_msg or "invest" in last_msg:
        sysint = MONEY_SYSINT
        llm_to_use = llm_money
        if "MA" not in state["active_advisors"] and "CA" in state["active_advisors"]:
            invite_needed = True
            state["active_advisors"].append("MA")
    else:
        sysint = CAREER_SYSINT + "\n" + MONEY_SYSINT  # Dual response for ambiguous or combined topics
        llm_to_use = llm_career  # Default to career LLM for dual responses
        if "CA" not in state["active_advisors"] or "MA" not in state["active_advisors"]:
            invite_needed = True
            if "CA" not in state["active_advisors"]:
                state["active_advisors"].append("CA")
            if "MA" not in state["active_advisors"]:
                state["active_advisors"].append("MA")
    
    message_history = [AIMessage(content=sysint)] + state["messages"]
    prompt = "\n".join([f"{m.type}: {m.content}" for m in message_history])
    
    # Handle local vs cloud LLM invocation
    if CONFIG["MODEL_CONFIG"]["career" if llm_to_use == llm_career else "money"]["type"] == "local":
        response = llm_to_use(prompt, max_tokens=CONFIG["MODEL_CONFIG"]["career" if llm_to_use == llm_career else "money"]["max_tokens"])
        advisor_response = response["choices"][0]["text"].strip()
    else:
        response = llm_to_use.invoke(prompt)
        advisor_response = response.content.strip()
    advisor_response = advisor_response.replace("ai:", "").strip()  # Remove stray "ai:" prefix
    
    # Check for tool calls
    if "retrieve_context" in advisor_response.lower():
        domain = "career" if "career" in last_msg else "money"
        query = f"{state[domain + '_goal']} {state['question']}" if state["question"] else state[domain + "_goal"]
        return state | {"messages": [AIMessage(content=advisor_response, tool_calls=[{"name": "retrieve_context", "args": {"query": query, "domain": domain}, "id": f"{domain}_mock_id"}])]}
    return state | {"messages": [AIMessage(content=advisor_response)]}

def context_node(state: AdvisorState) -> AdvisorState:
    last_msg = state["messages"][-1]
    outbound_msgs = []
    for tool_call in getattr(last_msg, "tool_calls", []):
        if tool_call["name"] == "retrieve_context":
            query = tool_call["args"]["query"]
            domain = tool_call["args"]["domain"]
            context = retrieve_context.invoke({"query": query, "domain": domain})
            if domain == "career":
                state["career_context"] = context
            elif domain == "money":
                state["money_context"] = context
            outbound_msgs.append(ToolMessage(content=context, name="retrieve_context", tool_call_id=tool_call["id"]))
    return state | {"messages": outbound_msgs}

def human_node(state: AdvisorState) -> AdvisorState:
    last_msg = state["messages"][-1]
    print(f"Advisor: {last_msg.content}")
    # Prompt for rating if evaluation is enabled (except welcome)
    if CONFIG["ENABLE_EVALUATION"] and len(state["messages"]) > 1 and isinstance(last_msg, AIMessage):
        while True:
            rating = input("Rate this advice (1-5, or 'skip'): ")
            if rating.lower() == "skip":
                break
            try:
                rating_int = int(rating)
                if 1 <= rating_int <= 5:
                    state["ratings"].append(rating_int)
                    break
                else:
                    print("Please enter a number between 1 and 5, or 'skip'.")
            except ValueError:
                print("Please enter a valid number (1-5) or 'skip'.")
    
    user_input = input("You: ")
    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True
        # Display evaluation summary and save ratings on exit
        if state["ratings"]:
            total_rated = len(state["ratings"])
            avg_rating = sum(state["ratings"]) / total_rated
            print("\nEvaluation Summary:")
            print(f"Total Responses Rated: {total_rated}")
            print(f"Average Rating: {avg_rating:.2f}/5")
            print(f"Ratings: {state['ratings']}")
            # Save ratings to JSON file with error handling
            ratings_data = {
                "session_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "ratings": state["ratings"],
                "average_rating": avg_rating,
                "total_responses": total_rated,
                "conversation": [msg.content for msg in state["messages"]]
            }
            output_file = "/kaggle/working/ratings.json"
            try:
                if os.path.exists(output_file):
                    with open(output_file, "r") as f:
                        existing_data = json.load(f)
                    if not isinstance(existing_data, list):
                        existing_data = [existing_data]
                else:
                    existing_data = []
                existing_data.append(ratings_data)
                with open(output_file, "w") as f:
                    json.dump(existing_data, f, indent=2)
                print(f"Ratings saved to {output_file}")
            except Exception as e:
                print(f"Error saving ratings: {e}")
        else:
            print("\nNo ratings provided.")
    return state | {"messages": [HumanMessage(content=user_input)]}

# Routing logic
def route_chat(state: AdvisorState) -> str:
    if state.get("finished", False):
        return END
    last_msg = state["messages"][-1]
    if isinstance(last_msg, ToolMessage):
        return "chatbot"
    if isinstance(last_msg, AIMessage) and hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "context"
    if isinstance(last_msg, HumanMessage):
        return "chatbot"
    return "human"

# Build the graph
workflow = StateGraph(AdvisorState)
workflow.add_node("chatbot", chatbot)
workflow.add_node("context", context_node)
workflow.add_node("human", human_node)

workflow.set_entry_point("chatbot")
workflow.add_conditional_edges("chatbot", route_chat)
workflow.add_conditional_edges("context", route_chat)
workflow.add_conditional_edges("human", route_chat)

# Compile the graph
app = workflow.compile()

# Test the interactive chat
initial_state = {
    "career_goal": "",
    "money_goal": "",
    "question": "",
    "career_context": "",
    "money_context": "",
    "combined_advice": {},
    "messages": [],
    "finished": False,
    "active_advisors": [],
    "ratings": []
}
config = {"recursion_limit": 100}
print("Starting interactive chat. Type 'q' to quit.")
state = app.invoke(initial_state, config)

Starting interactive chat. Type 'q' to quit.
Advisor: Welcome! I’m your Career Advisor (CA), and my colleague is your Money Advisor (MA). How can we assist you today?


You:  I want to be a comedian


Advisor: Career Advisor (CA): That's great!  To help you become a comedian, let's break it down. What's your current experience?  Are you aiming for stand-up, improv, sketch comedy, or something else?  What's your timeframe for achieving this goal?


You:  none


Advisor: Career Advisor (CA): Okay, no prior experience. That's a good starting point. To become a comedian with no experience, you'll need to build skills.  Do you prefer performing in front of an audience or writing jokes?  What kind of comedy are you drawn to?


...

Advisor: Career Advisor (CA): Great! Washington, DC has a vibrant comedy scene.  I recommend checking out venues like the DC Improv, Arlington Cinema & Drafthouse, and the Hotbed.  Look for open mic nights and workshops.  Also, search online for comedy groups and classes in the DC area.  A quick online search should provide many options.  Do you have any questions about finding these resources or getting started?


You:  no questions. thanks. let's finalize this plan


Advisor: ```json
{
  "career": "Become a comedian in Washington, DC.  Take improv classes, write jokes regularly, perform at open mic nights (DC Improv, Arlington Cinema & Drafthouse, Hotbed), join a comedy writing group. ",
  "money": "",
  "combined": "",
  "timestamp": "2024-07-26"
}
```


You:  quit



No ratings provided.


## Download ratings.json 
run this cell after chat ends

In [20]:
# Access ratings.json (run this cell after chat ends)
output_file = "/kaggle/working/ratings.json"
if os.path.exists(output_file):
    print(f"File exists at {output_file}, size: {os.path.getsize(output_file)} bytes")
    print("Download ratings.json from the output section below this cell after running.")
    # Fallback: Print contents
    with open(output_file, "r") as f:
        print("File contents (copy-paste if needed):")
        print(f.read())
else:
    print("No ratings.json file found. Run the chat and rate responses to generate it.")

File exists at /kaggle/working/ratings.json, size: 19671 bytes
Download ratings.json from the output section below this cell after running.
File contents (copy-paste if needed):
[
  {
    "session_date": "2025-04-07 20:08:57",
    "ratings": [
      2,
      3,
      4,
      4,
      4,
      3,
      1,
      5,
      2,
      2
    ],
    "average_rating": 3.0,
    "total_responses": 10,
    "conversation": [
      "Welcome! I\u2019m your Career Advisor (CA), and my colleague is your Money Advisor (MA). How can we assist you today?",
      "I am an 8 year old boy. I love playing video games. How can I do what I enjoy when I grow up and still make money?",
      "Money Advisor (MA): This seems like a career question\u2014let me bring in my colleague, the Career Advisor (CA), to assist.",
      "ok",
      "Career Advisor (CA): That's awesome you love video games!  There are many ways to combine your passion with a career.  To give you the best advice, could you tell me more about *wh

## Next Steps
* Add interactivity *[Done]*
* Fix text cut-off issue in advisor responses *[Done]*
* Feature: ability to use different LLM models for each advisor (both Gemini-1.5-flash, configurable) *[Done]*
* Feature: ability to switch models easily in the code *[Done]*
* Use @tool annotation for RAG instead of direct LLM calls *[Done]*
* Switched back to Gemini LLM with updated versions *[Done]*
* Updated initial message and system instructions *[Done]*
* Fixed repeated invites and unclear speaker labels *[Done]*
* Add evaluation section *[Done]*
* Feature: ability switch models easily in the code
* Polish for submission